Project final reports
======================================
#### XXX<br>Computer Sciences Department<br>University of Wisconsin-Madison
#### Decmber xx, 2023

### Cargo Distribution Route Optimization in California
## By: 
## Introduction
### Problem StatementOur project is the optimization of cargo distribution routes in California, where each city has warehouses, and dispatchable vehicles, and cargo must be transported by vehicles for inter-city transportation or co-located delivery, and where there are limitations on the number of vehicles that can be dispatched at the same time, and where the total cost of dispatching is minimized by planning reasonable dispatch routes to meet the demand in each city. Factors to be considered include the number of vehicles, the demand in each city, regular and overtime wages, and commuting time between cities.l. 

### Data:
Ten cities were randomly selected from California and the population size of each city was obtained. 

In [1]:
import pandas as pd

# Read the CSV file
file_path = 'us-cities-table.csv'
columns_to_read = ['name', 'pop2023', 'usps']
df = pd.read_csv(file_path, usecols=columns_to_read)

# Filter rows where usps='CA'
ca_data = df[df['usps'] == 'CA']

selected_rows = ca_data.sample(n=10)

# Sort by the 'pop2023' attribute in descending order
sorted_df = selected_rows.sort_values(by='pop2023', ascending=False)
print(sorted_df)

filtered_data = sorted_df[sorted_df['pop2023'] > 600000]
print(filtered_data)

     pop2023 usps           name
12    930862   CA       San Jose
32    549242   CA         Fresno
58    322538   CA      Riverside
102   219419   CA        Fremont
108   216456   CA  Moreno Valley
109   214718   CA        Fontana
120   201155   CA         Oxnard
169   153160   CA        Hayward
251   115761   CA       Carlsbad
257   114450   CA       Richmond
    pop2023 usps      name
12   930862   CA  San Jose


### Calculate commuting time between cities
With the ten cities previously obtained, the commuting time between cities is calculated through the api of goole map.With the ten cities previously obtained, the commuting time between cities is calculated through the api of goole map, and the results of the calculation will be prepared for the next step of the calculation

In [2]:
from serpapi import GoogleSearch

destination_cities = ['Los Angeles', 'San Francisco', 'Sacramento', 'San Jose', 'Oakland',
                      'Anaheim', 'Fresno', 'Long Beach', 'Santa Monica', 'San Diego']

common_params = {
    "engine": "google_maps_directions",
    "api_key": "668089e225ed59d57386d1df1d4f81b222afc963a4a3a1fb816f3997278d33e9",
    "travel_mode": "0"
}

The list of cities was obtained through previous calculations and accessed to the Google Maps api in preparation for calculating the commute time between cities

In [3]:
travel_time = [[0] * len(destination_cities) for _ in range(len(destination_cities))]

for i, city1 in enumerate(destination_cities):
    common_params["start_addr"] = f"{city1}, USA"

    for j, city2 in enumerate(destination_cities):
        common_params["end_addr"] = f"{city2}, USA"
        if city1 == city2:
            travel_time[i][j] = 0
            continue
        try:
            search = GoogleSearch(common_params)
            results = search.get_dict()
            directions = results.get("directions", [])
            if directions:
                travel_time[i][j] = round(int(directions[0]['duration'])/60/60, 2)
                print(directions)
            else:
                travel_time[i][j] = float('inf')

        except Exception as e:
            print(f"Error processing request for {city1} to {city2}: {e}")
            travel_time[i][j] = float('inf')

for row in travel_time:
    print(row)

[0, inf, inf, inf, inf, inf, inf, inf, inf, inf]
[inf, 0, inf, inf, inf, inf, inf, inf, inf, inf]
[inf, inf, 0, inf, inf, inf, inf, inf, inf, inf]
[inf, inf, inf, 0, inf, inf, inf, inf, inf, inf]
[inf, inf, inf, inf, 0, inf, inf, inf, inf, inf]
[inf, inf, inf, inf, inf, 0, inf, inf, inf, inf]
[inf, inf, inf, inf, inf, inf, 0, inf, inf, inf]
[inf, inf, inf, inf, inf, inf, inf, 0, inf, inf]
[inf, inf, inf, inf, inf, inf, inf, inf, 0, inf]
[inf, inf, inf, inf, inf, inf, inf, inf, inf, 0]


The results are calculated and stored in a table, so that the output is in a form that is easy to use in the next step, and the data represent this commuting time between one city and another, in trivial units

## Optimization Problem Approach
### Mixed-Integer Programming
MIP is a mathematical optimization method used to solve complex optimization problems that involve both integer and continuous variables. In MIP problems, the objective is to minimize or maximize a linear objective function while satisfying a set of linear equality and inequality constraints, with some variables required to be integers.The general form of a MIP problem can be represented as follows:

**Minimize (or Maximize):** Z = c1x1 + c2x2 + ... + cnxn

**Subject to constraints:**
a11x1 + a12x2 + ... + a1nxn <= b1
a21x1 + a22x2 + ... + a2nxn <= b2
...
am1x1 + am2x2 + ... + amnxn <= bm

Where,
- Z is the value of the objective function to be minimized (or maximized).
- x1, x2, ..., xn are the variables in the optimization problem, with some required to be integers.
- c1, c2, ..., cn are coefficients associated with the variables in the objective function.
- aij represents elements of the constraint matrix, describing relationships between constraints.
- b1, b2, ..., bm are the right-hand side values of the constraints.

MIP problems are typically solved using linear programming methods, but due to the presence of integer variables, they are often more challenging to solve than pure linear programming problems. MIP is a powerful mathematical tool used to address optimization problems involving both integer and continuous variables, with widespread applications across various domains.

## Model building:

#### Sets:
- `i`: Cities, \(i \in \{i1, i2, ..., i10\}\)
- `k`: Vehicles, \(k \in \{k1, k2, ..., k5\}\)
- `ij(i,j)`: City pairs

#### Scalars:
- `max_capacity`: Maximum capacity of vehicles (e.g., 35)
- `base_transportation_cost`: Base transportation cost (e.g., 50)
- `overtime_cost`: Overtime cost (e.g., 100)

#### Parameters:
- `demand(i)`: Demand for each city
- `work_time(i,j)`: Work time between cities
- `max_working_hours(k)`: Maximum working hours f
time\_cost} \right)

Customized parameters such as the demand of the city, the maximum carrying capacity of the vehicle, the number of vehicles that can be dispatched at the same time

### Objective Function
The objective function is to minimize the total cost, which includes the base transportation cost and overtime cost for each route.

$$
\text{Objective Function (obj)}: \min \left( \sum_{i,j,k} \left( \text{work\_time}(i, j) \cdot x(i, j, k) \cdot \text{base\_transportation\_cost} + \max(0, \text{work\_time}(i, j) - \text{max\_working\_hours}(k)) \cdot x(i, j, k) \cdot \text{overtime\_cost} \right) \right)
$$

### Constraints
#### Demand Constraint:
$$
\sum_{j,k} x(i, j, k) = \sum_{jj,k} x(jj, i, k) \quad \forall i
$$


#### Continuous Route Constraint:
$$ \sum_j (y(i, j, k) - y(j, i, k) + x(i, j, k) - x(j, i, k)) = \sum_j y(i, j, k) - \sum_j y(j, i, k) \quad \forall i \neq k $$


#### No Self-Route Constraint:
$$ x(i, i, k) = 0 \quad \forall i, k \quad \text{where } i = k $$


#### At Least One Route Constraint:
$$ \sum_k \sum_j x(i, j, k) \geq 1 \quad \forall i $$


#### Capacity Constraint:
$$ \sum_{i,j} \text{demand}(i) \cdot x(i, j, k) \leq \text{max\_capacity} \quad \forall k $$

In [4]:
from gams import GamsWorkspace
from gams import *

ws = GamsWorkspace(debug=DebugLevel.KeepFiles)

gams_code = '''
Sets
  i 'cities'    /i1*i10/
  k 'vehicles'  /k1*k5/
  ij(i,i) 'city pairs'
;

Alias(i,j,jj);

ij(i,j) = yes$(ord(i) <> ord(j));

Scalars
max_capacity 'maximum capacity' /35/
base_transportation_cost 'base transportation cost' /50/
overtime_cost 'overtime cost' /100/
;

Parameters
  demand(i) 'demand for each city'
  /i1 15, i2 1, i3 8, i4 14, i5 4, i6 14, i7 12, i8 16, i9 15, i10 11/
;

parameter work_time(i,j) 'work time between cities'
  / i1.i2  6.19, i1.i3  5.74, i1.i4  5.45, i1.i5  5.78, i1.i6  0.44, i1.i7  3.38, i1.i8  0.41, i1.i9  0.33, i1.i10 1.77
    i2.i1  6.19, i2.i3  1.36, i2.i4  0.79, i2.i5  0.26, i2.i6  6.39, i2.i7  2.96, i2.i8  6.39, i2.i9  6.02, i2.i10 8.03
    i3.i1  5.74, i3.i2  1.36, i3.i4  1.96, i3.i5  1.31, i3.i6  6.44, i3.i7  2.73, i3.i8  6.46, i3.i9  6.1,  i3.i10 8.09
    i4.i1  5.45, i4.i2  0.79, i4.i3  1.96, i4.i5  0.69, i4.i6  5.69, i4.i7  2.47, i4.i8  5.33, i4.i9  5.0,  i4.i10 6.75
    i5.i1  5.78, i5.i2  0.26, i5.i3  1.31, i5.i4  0.69, i5.i6  6.16, i5.i7  2.79, i5.i8  6.17, i5.i9  5.8,  i5.i10 7.75
    i6.i1  0.44, i6.i2  6.39, i6.i3  6.44, i6.i4  5.69, i6.i5  6.16, i6.i7  3.77, i6.i8  0.44, i6.i9  0.62, i6.i10 1.42
    i7.i1  3.38, i7.i2  2.96, i7.i3  2.73, i7.i4  2.47, i7.i5  2.79, i7.i6  3.77, i7.i8  3.77, i7.i9  3.43, i7.i10 5.15
    i8.i1  0.41, i8.i2  6.39, i8.i3  6.46, i8.i4  5.33, i8.i5  6.17, i8.i6  0.44, i8.i7  3.77, i8.i9  0.49, i8.i10 1.63
    i9.i1  0.33, i9.i2  6.02, i9.i3  6.1,  i9.i4  5.0,  i9.i5  5.8,  i9.i6  0.62, i9.i7  3.43, i9.i8  0.49, i9.i10 1.96
    i10.i1 1.77, i10.i2 8.03, i10.i3 8.09, i10.i4 6.75, i10.i5 7.75, i10.i6 1.42, i10.i7 5.15, i10.i8 1.63, i10.i9 1.96
/;

parameter
max_working_hours(k)
/k1   8
k2    8
k3    8
k4    8
k5    8
/;

Binary Variables
x(i,j,k) 'binary decision variable for routes'
y(i,j,k) 'binary decision variable for connected routes'
;

Variables
obj  'objective function';

Equations
objective 'objective function'
demand_constraint(i) 'demand constraint for each city'
capacity_constraint(k) 'capacity constraint for each vehicle'
no_self_route(i,k) 'constraint for no self-route'
at_least_one_route(i) 'constraint to ensure each city is visited at least once'
continuous_route(i,k) 'The route continuity constraint for each vehicle'
;

objective ..
obj =e= sum((i,j,k)$(ord(i) <> ord(j)),
    work_time(i,j) * x(i,j,k) * base_transportation_cost
    + max(0, work_time(i,j) - max_working_hours(k)) * x(i,j,k) * overtime_cost
  );

demand_constraint(i) ..
sum( (j,k)$((ord(j) <> ord(i))),x(i,j,k) )
  =E= sum( (jj,k)$((ord(jj) <> ord(i))),x(jj,i,k) );
  
continuous_route(i,k)$(ord(i) <> ord(k))..
    sum(j, y(i,j,k) - y(j,i,k) + x(i,j,k) - x(j,i,k)) =E= sum(j, y(i,j,k)) - sum(j, y(j,i,k));


no_self_route(i,k)$(ord(i) = ord(k))..
    x(i, i, k) =e= 0;

at_least_one_route(i)..
    sum(k, sum(j$(ord(j)<>ord(i)), x(i,j,k))) =G= 1;


capacity_constraint(k) ..
sum( (i, j)$(ord(i) <> ord(j)),
    demand(i) * x(i,j,k) ) =L= max_capacity;
    

Model DeliveryOptimization /all/;

Solve DeliveryOptimization using MIP minimizing obj;

Display obj.L, x.L, x.M;

'''

job = ws.add_job_from_string(gams_code)
job.run()

result_obj = job.out_db['obj']
result_x = job.out_db['x']

result_obj_record = result_obj.first_record()
print("Objective value:", result_obj_record.level,"$")

# print("\nVariable x values:")
# for rec in result_x:
#     if rec.get_level() != 0.0:  
#         print(rec.key(0), rec.key(1), rec.key(2))

organized_data = {"k1": [], "k2": [], "k3": [], "k4": [], "k5": []}

for rec in result_x:
    if rec.get_level() != 0.0:  
        key = rec.key(2)  
        value = rec.key(0)  
        organized_data[key].append(value)  

for key in organized_data:
    organized_data[key].append(organized_data[key][0])

for key in sorted(organized_data.keys()):
    values = '-> '.join([f'city{v[1:]}' for v in organized_data[key]])
    print(f"vehicles {key}: {values}")


Objective value: 576.0 $
vehicles k1: city2-> city4-> city5-> city2
vehicles k2: city3-> city7-> city3
vehicles k3: city1-> city8-> city1
vehicles k4: city6-> city10-> city6
vehicles k5: city1-> city9-> city1


### Visualization
Draw a road map for dispatching vehicles, with each of the five lines representing the routes of five vehicles.

In [25]:
import webbrowser
import folium

formatted_values = []

for key in sorted(organized_data.keys()):
    
    values = organized_data[key]

    formatted_values.append([values])

# City names
cities = ['Los Angeles', 'San Francisco', 'Sacramento', 'San Jose', 'Oakland',
          'Anaheim', 'Fresno', 'Long Beach', 'Santa Monica', 'San Diego']

# Coordinates for each city
cities_points = {
    'Los Angeles': (34.0522, -118.2437),
    'San Francisco': (37.7749, -122.4194),
    'Sacramento': (38.5816, -121.4944),
    'San Jose': (37.3382, -121.8863),
    'Oakland': (37.8049, -122.2711),
    'Anaheim': (33.8366, -117.9143),
    'Fresno': (36.7783, -119.4179),
    'Long Beach': (33.7701, -118.1937),
    'Santa Monica': (34.0242, -118.4965),
    'San Diego': (32.7157, -117.1611)
}


# Convert data to coordinates
coordinates_data = []

for sublist in formatted_values:
    coordinates_sublist = [(cities_points[cities[int(city[1:]) - 1]][0], cities_points[cities[int(city[1:]) - 1]][1]) for city in sublist[0]]
    coordinates_data.append(coordinates_sublist)

print(coordinates_data)


# Function to plot routes on a map
def plot_routes_on_map(routes_waypoints, filename='routes_map.html'):
    my_map = folium.Map(location=[routes_waypoints[0][0][0], routes_waypoints[0][0][1]], zoom_start=4)
    colors = ['blue', 'green', 'red', 'orange', 'purple', 'brown', 'pink']
    for route_id, waypoints in enumerate(routes_waypoints):
        color = colors[route_id % len(colors)]  # Cycle through the color list
        popup_text = f'Route {route_id + 1}'
        folium.PolyLine(locations=waypoints, color=color, weight=2.5, opacity=1, popup=popup_text).add_to(my_map)
        for point_id, point in enumerate(waypoints):
            folium.Marker(location=point, icon=folium.Icon(color=color),
                          popup=f'Point {point_id + 1}').add_to(my_map)
            folium.Marker(location=point,
                          icon=folium.DivIcon(html=f'<div style="font-size: 12; color: black;">{point_id + 1}</div>'),
                          popup=f'Point {point_id + 1}').add_to(my_map)
    my_map.save(filename)
    webbrowser.open(filename)

plot_routes_on_map(coordinates_data, filename='routes_map.html')

[[(37.7749, -122.4194), (37.3382, -121.8863), (37.8049, -122.2711), (37.7749, -122.4194)], [(38.5816, -121.4944), (36.7783, -119.4179), (38.5816, -121.4944)], [(34.0522, -118.2437), (33.7701, -118.1937), (34.0522, -118.2437)], [(33.8366, -117.9143), (32.7157, -117.1611), (33.8366, -117.9143)], [(34.0522, -118.2437), (34.0242, -118.4965), (34.0522, -118.2437)]]


![picture](roadmap.png)

## Conclusion
Inter-city transportation or delivery of goods by dispatching vehicles from four cities: City 1, City 2, City 3 and City 6.The final minimum cost is $576 for five vehicles departing from each of the four warehouses and returning to the departing warehouse

In [26]:
result_obj_record = result_obj.first_record()
print("Objective value:", result_obj_record.level,"$")

for key in sorted(organized_data.keys()):
    values = '-> '.join([f'city{v[1:]}' for v in organized_data[key]])
    print(f"vehicles {key}: {values}")

Objective value: 576.0 $
vehicles k1: city2-> city4-> city5-> city2
vehicles k2: city3-> city7-> city3
vehicles k3: city1-> city8-> city1
vehicles k4: city6-> city10-> city6
vehicles k5: city1-> city9-> city1


## further extension
The next step can be to study the VRP with Time Windows (VRPTW) problem with the addition of the time window factor, where each customer has a specified time window for service, i.e., it specifies the timeframe within which the customer can receive service. The goal is to find a set of vehicle paths that will satisfy all customer needs and complete the service within each customer's time window. This factor is not currently taken into account